# **Draft**

In [1]:
# nn stuff
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
# helper stuff
import numpy as np
import math
import h5py
import matplotlib.pyplot as plt
import scipy

In [21]:
"""Variables"""
# names
DATA = "dataset.hdf5"
x_GroundTruth = [] # this is our Y
P_SamplingOperator = [] # [] is how u initialize array (to nothing)
x_hat_ZeroFilledImg = [] # this is our x (input to NN)
y_NoisyMeasurement = []
e = 0
"""dividing data"""
data_total = 360 # got 360 images total
# training part
train_total = 300
train_ct = 294
validate_ct = 6 # train_ct + validate_ct = train_total
dev_ct = 32 # for development
# testing part
test_total = 60
test_ct = 6 # test_ct = (1/10)test_total


In [22]:
# load data
with h5py.File(DATA, 'r') as hdf:
    keys = list(hdf.keys())
    trnOrg = hdf.get('trnOrg')
    x_GroundTruth = np.array(trnOrg)
    trnMask = hdf.get('trnMask')
    P_SamplingOperator = np.array(trnMask)

In [23]:
"""calculate y"""
# Formula: y = P*Fourier(x) + e
y_NoisyMeasurement = P_SamplingOperator * np.fft.fft2(x_GroundTruth) + e
"""calculate x_hat"""
# Formula: x_hat = InverseFourier(y)
x_hat_ZeroFilledImg = np.fft.ifft2(y_NoisyMeasurement)

In [24]:
"""divide data to: training & testing set"""
# Reference: cs231n, Assignment 1, svm.ipynb
train_SET_mask = range(train_total)
X_SET_TRAIN = x_hat_ZeroFilledImg[train_SET_mask]
Y_SET_TRAIN = x_GroundTruth[train_SET_mask]

test_SET_mask = range(train_total, train_total + test_total)
X_SET_TEST = x_hat_ZeroFilledImg[test_SET_mask]
Y_SET_TEST = x_GroundTruth[test_SET_mask]

train_mask = range(train_ct)
x_train = X_SET_TRAIN[train_mask]
y_train = Y_SET_TRAIN[train_mask]

validate_mask = range(train_ct, train_ct + validate_ct)
x_val = X_SET_TRAIN[validate_mask]
y_val = Y_SET_TRAIN[validate_mask]

dev_mask = np.random.choice(train_ct, dev_ct, replace=False)
x_dev = X_SET_TRAIN[dev_mask]
y_dev = Y_SET_TRAIN[dev_mask]

test_mask = range(test_ct)
x_test = X_SET_TEST[test_mask]
y_test = Y_SET_TEST[test_mask]

In [29]:
"""preprocess"""
# turn each image into rows
x_train = np.reshape(x_train, (x_train.shape[0], -1))
x_val = np.reshape(x_val, (x_val.shape[0], -1))
x_dev = np.reshape(x_dev, (x_dev.shape[0], -1))
x_test = np.reshape(x_test, (x_test.shape[0], -1))
# preprocess: subtract mean
mean_img = np.mean(x_train, axis=0)
x_train -= mean_img
x_val -= mean_img
x_dev -= mean_img
x_test -= mean_img

### **U-Net description**
Source: https://arxiv.org/pdf/1505.04597.pdf
<br>It consists of a contracting path (left side) and an expansive path (right side). 

The contracting path follows the typical architecture of a convolutional network. 
>It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), 
><br>each followed by a rectified linear unit (ReLU) 
><br>and a 2x2 max pooling operation with stride 2 for downsampling. 
><br>At each downsampling step we double the number of feature channels. 

Every step in the expansive path consists of 
>an upsampling of the feature map followed by a 2x2 convolution (“up-convolution”) that halves the number of feature channels, 
><br>a concatenation with the correspondingly cropped feature map from the contracting path, 
><br>and two 3x3 convolutions, each followed by a ReLU. 
><br>The cropping is necessary due to the loss of border pixels in every convolution. 

At the final layer a 1x1 convolution is used to map each 64-component feature vector to the desired number of classes. In total the network has 23 convolutional layers.

In [31]:
"""Neural Network"""
import nn_helper
# Reference:
# https://arxiv.org/pdf/1505.04597.pdf
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(NeuralNetwork, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            ######################
            # Contracting step
            ######################
            # two 3x3 convolutions, unpadded, each followed by rectified linear unit
            # in_channels = 2, because MRI image has real and complex parts
            # (?) out_channels = 2, because output and input images have same format
            # Question: stride = 1 ?
            nn.Conv2d(in_channels=n_channels, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            # followed by 2x2 max pooling, stride = 2, for downsampling
            # Question: no padding?
            nn.MaxPool2d(kernel_size=(2,2), stride=2),

            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),

            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),

            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            ######################
            # Bottom part
            ######################
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            ######################
            # Expanding part
            ######################
            # upsampling of the feature map,
            # followed by a 2x2 (up-)convolution that halves the feature channels 
            # (?) a concatenation with the correspondingly cropped feature map from the contracting path
            # and two 3x3 convolutions, each followed by a ReLU
            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),

            nn.Upsample(scale_factor=2, mode='bilinear'),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1),
            nn.ReLU(),

            ######################
            # Output part
            ######################
            # At the final layer a 1x1 convolution is used
            nn.Conv2d(in_channels=2, out_channels=2, kernel_size=(3, 3), stride=1)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(2,2).to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (11): ReLU()
    (12): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (16): ReLU()
    (17): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1))
    (18): ReLU()
    (19): MaxPool2d(kernel_